# Исследование уровня потребительской лояльности

Заказчик этого исследования — большая телекоммуникационная компания, которая оказывает услуги на территории всего СНГ. Перед компанией стоит задача определить текущий уровень потребительской лояльности, или NPS (от англ. Net Promoter Score), среди клиентов из России. 

Чтобы определить уровень лояльности, клиентам задавали классический вопрос: «Оцените по шкале от 1 до 10 вероятность того, что вы порекомендуете компанию друзьям и знакомым».

Компания провела опрос и попросила нас подготовить дашборд с его итогами. Большую базу данных для такой задачи разворачивать не стали и выгрузили данные в SQLite. 

Чтобы оценить результаты опроса, оценки обычно делят на три группы:
- 9-10 баллов — «cторонники» (англ. promoters);
- 7-8 баллов — «нейтралы» (англ. passives);
- 0-6 баллов — «критики» (англ. detractors).

Итоговое значение NPS рассчитывается по формуле: % «сторонников» - % «критиков».

Таким образом, значение этого показателя варьируется от -100% (когда все клиенты «критики») до 100% (когда все клиенты лояльны к сервису). Но это крайние случаи, которые редко встретишь на практике. 

Интерпретируя результаты NPS-опросов, следует также помнить, что само значение мало о чём говорит. Однако исследования показывают, что клиенты-сторонники полезны любому бизнесу. Они чаще других повторно совершают покупки, активнее тестируют обновления и приводят в сервис своих друзей и знакомых. Поэтому NPS остаётся одной из важнейших метрик бизнеса. 

## Подключение к базе данных

Прежде чем начать работу, нам необходимо получить доступ к базе данных. Данные выгрузили в SQLite — СУБД, в которой база данных представлена файлом.

In [1]:
# Подключение всех необходимых библиотек
import os
import pandas as pd

# Библиотека для создания ссылки
from IPython.display import FileLink

from sqlalchemy import create_engine

In [2]:
# Путь к БД на платформе
path_to_db_platform = '/datasets/telecomm_csi.db'
path_to_db = None

# Если путь на платформе ведёт к БД, то он становится итоговым
if os.path.exists(path_to_db_platform):
    path_to_db = path_to_db_platform
# Иначе выводится сообщение о том, что файл не найден
else:
    raise Exception('Файл с базой данных SQLite не найден!')

# Если итоговый путь не пустой
if path_to_db:
    # То создаём подключение к базе
    engine = create_engine(f'sqlite:///{path_to_db}', echo=False)
    print("Подключение к базе данных создано!")
# Иначе выводится сообщение о том, что подключение не создано
else:
     raise Exception('Создать подключение не удалось!')

Подключение к базе данных создано!


Мы подключились к базе данных. Теперь эти данные необходимо выгрузить.

## Выгрузка данных

Нужно подготовить данные для дашборда.

Нам нужно собрать в одну витрину данные из разных таблиц. Эту витрину мы будем использовать для дашборда.

Итак, у нас есть 5 таблиц `user`, `location`, `age_segment`, `traffic_segment`и `lifetime_segment`.<br>
Напишем SQL-запрос, чтобы соединить эти таблицы в одну. Учтем все требования.

In [3]:
query = """
SELECT 
    u.user_id,
    u.lt_day,
    CASE 
        WHEN u.lt_day <= 365 THEN 1 
        ELSE 0 
    END AS is_new,
    CAST(u.age AS INTEGER) AS age,  -- Преобразование возраста в целое число
    CASE 
        WHEN u.gender_segment = 1 THEN 'Женщина'
        WHEN u.gender_segment = 0 THEN 'Мужчина'
        ELSE 'Неопределенный'
    END AS gender_segment,
    u.os_name,
    u.cpe_type_name,
    l.country,
    l.city,
    SUBSTR(a.title, INSTR(a.title, ' ') + 1) AS age_segment,  -- Извлечение возрастного диапазона
    SUBSTR(t.title, INSTR(t.title, ' ') + 1) AS traffic_segment,  -- Извлечение диапазона трафика
    SUBSTR(lt.title, INSTR(lt.title, ' ') + 1) AS lifetime_segment,  -- Извлечение диапазона дней жизни
    u.nps_score,
    CASE 
        WHEN u.nps_score >= 9 THEN 'promoters'
        WHEN u.nps_score >= 7 THEN 'passives'
        ELSE 'detractors'
    END AS nps_group
FROM 
    user AS u
JOIN 
    location AS l ON u.location_id = l.location_id
JOIN 
    age_segment AS a ON u.age_gr_id = a.age_gr_id
JOIN 
    traffic_segment AS t ON u.tr_gr_id = t.tr_gr_id
JOIN 
    lifetime_segment AS lt ON u.lt_gr_id = lt.lt_gr_id;
"""

df = pd.read_sql(query, engine)

In [4]:
df.head()

,user_id,lt_day,is_new,age,gender_segment,os_name,cpe_type_name,country,city,age_segment,traffic_segment,lifetime_segment,nps_score,nps_group
0,A001A2,2320,0,45.0,Женщина,ANDROID,SMARTPHONE,Россия,Уфа,45-54,1-5,36+,10,promoters
1,A001WF,2344,0,53.0,Мужчина,ANDROID,SMARTPHONE,Россия,Киров,45-54,1-5,36+,10,promoters
2,A003Q7,467,0,57.0,Мужчина,ANDROID,SMARTPHONE,Россия,Москва,55-64,20-25,13-24,10,promoters
3,A004TB,4190,0,44.0,Женщина,IOS,SMARTPHONE,Россия,РостовнаДону,35-44,0.1-1,36+,10,promoters
4,A004XT,1163,0,24.0,Мужчина,ANDROID,SMARTPHONE,Россия,Рязань,16-24,5-10,36+,10,promoters


### Проверка корректности данных

Проверим корректность числовых полей.

In [5]:
df['age'].describe()

count    501939.000000
mean         39.621946
std          11.188249
min          10.000000
25%          31.000000
50%          38.000000
75%          47.000000
max          89.000000
Name: age, dtype: float64

In [6]:
df['nps_score'].describe()

count    502493.000000
mean          7.508562
std           3.020378
min           1.000000
25%           5.000000
50%           9.000000
75%          10.000000
max          10.000000
Name: nps_score, dtype: float64

In [7]:
df['lt_day'].describe()

count    502493.000000
mean       1868.841439
std        1683.701762
min         -21.000000
25%         533.000000
50%        1239.000000
75%        3064.000000
max        9162.000000
Name: lt_day, dtype: float64

Посмотрим, сколько в этом столбце отрицательных значений.

In [8]:
df.query('lt_day < 0')

,user_id,lt_day,is_new,age,gender_segment,os_name,cpe_type_name,country,city,age_segment,traffic_segment,lifetime_segment,nps_score,nps_group
7666,AEAC2R,-8,1,NaN,Неопределенный,ANDROID,SMARTPHONE,Россия,Томск,n/a,15-20,36+,1,detractors
53997,CS0HF8,-2,1,NaN,Неопределенный,ANDROID,SMARTPHONE,Россия,Ижевск,n/a,20-25,36+,10,promoters
71577,DORPT2,-4,1,NaN,Неопределенный,ANDROID,SMARTPHONE,Россия,Москва,n/a,20-25,36+,4,detractors
100152,F5O3CG,-21,1,NaN,Неопределенный,ANDROID,SMARTPHONE,Россия,Чита,n/a,35-40,36+,7,passives
228087,LSE939,-13,1,NaN,Неопределенный,ANDROID,SMARTPHONE,Россия,Краснодар,n/a,15-20,36+,1,detractors
325212,QTT7IR,-13,1,NaN,Неопределенный,IOS,SMARTPHONE,Россия,Москва,n/a,10-15,36+,7,passives
347784,RZUS6H,-6,1,NaN,Неопределенный,ANDROID,SMARTPHONE,Россия,Красноярск,n/a,30-35,36+,3,detractors
385897,TYWQW4,-12,1,NaN,Неопределенный,IOS,SMARTPHONE,Россия,Новосибирск,n/a,1-5,36+,5,detractors
401226,URKAFI,-11,1,NaN,Неопределенный,ANDROID,SMARTPHONE,Россия,НижнийНовгород,n/a,10-15,36+,8,passives
407160,V2GNK2,-13,1,NaN,Неопределенный,IOS,SMARTPHONE,Россия,РостовнаДону,n/a,15-20,36+,5,detractors


В этих строчках так же отсутствуют данные о возрасте и поле. Поскольку их незначительное количество, не будем брать их в финальный датасет. 

In [9]:
df = df.query('lt_day > 0')

In [10]:
df['lt_day'].describe()

count    502476.000000
mean       1868.904911
std        1683.694880
min           1.000000
25%         533.000000
50%        1239.000000
75%        3064.000000
max        9162.000000
Name: lt_day, dtype: float64

In [11]:
df['is_new'].describe()

count    502476.000000
mean          0.171318
std           0.376787
min           0.000000
25%           0.000000
50%           0.000000
75%           0.000000
max           1.000000
Name: is_new, dtype: float64

Числовые столбцы выглядят корректно.<br>
Теперь проверим, что в поле `city` отсутствуют неявные дубликаты городов.<br>
Будем группировать с `user_id`, поскольку это уникальное значение.

In [12]:
# Создание копии датасета
df_copy = df.copy()

# Приведение названий городов к единому формату (все строчные буквы и удаление лишних пробелов)
df_copy['city_clean'] = df['city'].str.lower().str.strip()

# Поиск дубликатов
duplicate_names = df_copy[df_copy.duplicated(subset=['user_id', 'city_clean'], keep='first')]
duplicate_names

,user_id,lt_day,is_new,age,gender_segment,os_name,cpe_type_name,country,city,age_segment,traffic_segment,lifetime_segment,nps_score,nps_group,city_clean


Неявных дублей не обнаружено.

### Сохранение данных

In [13]:
# Получившуюся таблицу сохраним как CSV-файл.
df.to_csv('telecomm_csi_tableau.csv', index=False)

Теперь создадим ссылку для локального скачивания файла, чтобы дальше использовать его в Tableau.

In [14]:
# Создание ссылки на скачивание файла
file_name = 'telecomm_csi_tableau.csv'
FileLink(file_name)

/home/jovyan/work/telecomm_csi_tableau.csv

## Создание дашборда в Tableau

Нам нужно построить дашборд, который представит информацию о текущем уровне NPS среди клиентов и покажет, как этот уровень меняется в зависимости от пользовательских признаков. 

Из дашборда должно быть понятно, какие группы пользователей участвовали в опросе. Обратим внимание, что если пользователи какой-то группы преобладают, то это могло отразиться на общем уровне NPS. 

Используя дашборд, ответим на вопросы:
- Как распределены участники опроса по возрасту и полу? Каких пользователей больше: новых или старых? Пользователи из каких городов активнее участвовали в опросе?
- Какие группы пользователей наиболее лояльны к сервису? Какие менее?
- Какой общий NPS среди всех опрошенных?
- Как можно описать клиентов, которые относятся к группе cторонников (англ. promoters)?

Ссылка на презентацию в Tableau Public: <https://public.tableau.com/app/profile/andrey.semenov/viz/telecomm_17221871544090/sheet22>